In [330]:
import duckdb
import geopandas as gpd
import jenkspy
from lonboard import BitmapTileLayer, Map, PolygonLayer
from lonboard.colormap import apply_categorical_cmap
import numpy as np
import pyarrow as pa

In [2]:
con = duckdb.connect()
con.install_extension("spatial")
con.load_extension("spatial")

# 1.0 Total private dwellings and private dwellings per square kilometer for Ottawa
These values are from the 2021 Census of Population

In [345]:
con.execute("""
DROP TABLE IF EXISTS geo_data;
CREATE TABLE geo_data AS
SELECT
    geo.da_dguid,
    cop.count_total_1,
    cop.count_total_4,
    cop.count_total_6,
    cop.count_total_7,
    ROUND(
        (cop.count_total_4 / cop.count_total_7), 2
    ) AS count_total_4_per_square_km,
    geo.geom
FROM
    'https://data.dataforcanada.org/processed/statistics_canada/census_of_population/2021/tabular/da_2021.parquet' AS cop,
    'https://data.dataforcanada.org/processed/statistics_canada/boundaries/2021/digital_boundary_files/da_2021.parquet' AS geo
WHERE geo.csd_name IN ('Ottawa') AND cop.da_dguid = geo.da_dguid;

""")

con.execute("""
COPY geo_data TO './da_2021_private_dwellings.parquet' (FORMAT PARQUET);
""")

In [346]:
private_dwellings_per_square_km = con.execute("SELECT DISTINCT count_total_4_per_square_km FROM geo_data").fetchall()

values = np.array([v[0] for v in private_dwellings_per_square_km])

# Compute Jenks breaks
num_classes = 5
breaks = jenkspy.jenks_breaks(values, n_classes=num_classes)

In [347]:
# Create a bin range mapping: (lower, upper) for each bin
bin_ranges = [(breaks[i], breaks[i+1]) for i in range(len(breaks)-1)]

# Create a function to get the range string for a value
def jenks_range(value) -> str:
    for i, (low, high) in enumerate(bin_ranges):
        if low <= value <= high:
            return f"{int(low)}–{int(high)}"
    return "unknown"


dwellings_df = gpd.read_parquet('./da_2021_private_dwellings.parquet')
dwellings_df['category'] = dwellings_df["count_total_4_per_square_km"].apply(lambda v: jenks_range(v))
dwellings_df['category'] = dwellings_df['category'].astype('category')

In [353]:
# Categories to colors
cmap = {}
colors = [
    [255, 255, 255],
    [255, 191.25, 191.25],
    [255, 127.50, 127.50],
    [255, 63.75, 63.75],
    [255, 0, 0]
]
for index, value in enumerate(dwellings_df['category'].unique()):
    cmap[value] = colors[index]

In [355]:
# OpenStreetMap

# We set `max_requests < 0` because `tile.openstreetmap.org` supports HTTP/2.
basemap = BitmapTileLayer(
    data="https://tile.openstreetmap.org/{z}/{x}/{y}.png",
    tile_size=256,
    max_requests=-1,
    min_zoom=0,
    max_zoom=19,
)

In [356]:
# Google Satellite
basemap = BitmapTileLayer(
    data="http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}",
    tile_size=256,
    max_requests=-1,
    min_zoom=0,
    max_zoom=19,
)

In [357]:
get_color = apply_categorical_cmap(pa.array(dwellings_df['category']), cmap)


In [358]:
cop_layer = PolygonLayer.from_geopandas(gdf=gpf,
                                        stroked=True,
                                        get_fill_color=get_color,
                                        get_line_color=[255, 255, 255],
                                        get_line_width=5,
                                        line_width_units="meters",
                                        opacity=0.4,
                                        auto_highlight = True,
                                        highlight_color=[0,0,0,0]
                                       )

In [359]:
m = Map([basemap, cop_layer])

In [360]:
m

Map(custom_attribution='', layers=(BitmapTileLayer(data='http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={…